In [1]:
import pandas as pd
import numpy as np
import nltk
import re

# import warnings
# warnings.filterwarnings('ignore')

## Загрузка данных

In [2]:
data = pd.read_csv('test_data.csv')
data.head()

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,manager,Ага


## Обработка данных

__Избавляемся от stopwords__

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Роман\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Роман\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Роман\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Роман\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Роман\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
stop_words = set(stopwords.words('russian'))
unnes_words = set({
    'угу',
    'ага',
    'алло'
})
regex = re.compile('[^a-я А-Я]')
preprocess_texts = []

for i in range(len(data)):
    text = data.text.iloc[i].lower()
    text = regex.sub(' ', text)
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words | unnes_words] 
    preprocess_texts.append(' '.join(filtered_sentence))

__Стемминг__

In [5]:
from nltk.stem.snowball import SnowballStemmer

In [6]:
st = SnowballStemmer(language='russian')
stem_text = []
for i in range(len(data)):
    word_tokens = word_tokenize(preprocess_texts[i])
    stem_text.append(' '.join([st.stem(word) for word in word_tokens]))

In [7]:
data['stem_processed_text'] = stem_text
# data_manager = data[data.role == 'client']

## Парсинг текста

__a) Извлекаем реплики с приветсвие - где менеджер поздоровался__

In [8]:
id_greeting = data['stem_processed_text'].str.contains('здравств|(добр (ден|утр|ноч))|добр')
data['insight'] = np.where(id_greeting, 'greeting=True', None)
data[id_greeting]

,dlg_id,line_n,role,text,stem_processed_text,insight
1,0,1,client,Алло здравствуйте,здравств,greeting=True
2,0,2,manager,Добрый день,добр ден,greeting=True
109,1,0,manager,Да здравствуйте когда заканчивается,здравств заканчива,greeting=True
110,1,1,client,Алло здравствуйте,здравств,greeting=True
165,2,1,manager,Здравствуйте,здравств,greeting=True
166,2,2,client,Алло здравствуйте,здравств,greeting=True
249,3,0,manager,Добрый день,добр ден,greeting=True
250,3,1,client,Алло дмитрий добрый день,дмитр добр ден,greeting=True
251,3,2,client,Добрый меня максим зовут компания китобизнес у...,добр макс зовут компан китобизнес удобн говор,greeting=True
300,3,51,client,Угу все хорошо да понедельника тогда всего доб...,понедельник добр,greeting=True


__b) Извлекаем реплики, где менеджер представил себя__

In [9]:
id_name = data['stem_processed_text'].str.contains('зовут')
data.insight[id_name] = 'introducing=True'
data[id_name]

,dlg_id,line_n,role,text,stem_processed_text,insight
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True
111,1,2,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True
167,2,3,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True
251,3,2,client,Добрый меня максим зовут компания китобизнес у...,добр макс зовут компан китобизнес удобн говор,introducing=True


In [10]:
data.loc[251, 'insight'] = 'introducing=True, greeting=True'

__c) Извлекаем имя менеджера__

In [11]:
from yargy import rule, and_, Parser, or_
from yargy.predicates import gram, eq

In [12]:
NAME = or_(
    rule(eq('зовут'), gram('Name')),
    rule(gram('Name'), eq('зовут'))
)

R_name = rule(NAME)

parser = Parser(R_name)

In [13]:
data['name'] = None

for i in data[id_name].index:
    for match in parser.findall(data['text'][i]):
        res = [_.value for _ in match.tokens]
        if res[0] == 'зовут':
            data['name'][i] = res[1]
        else: data['name'][i] = res[0]

In [14]:
data[id_name]

,dlg_id,line_n,role,text,stem_processed_text,insight,name
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True,ангелина
111,1,2,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True,ангелина
167,2,3,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True,ангелина
251,3,2,client,Добрый меня максим зовут компания китобизнес у...,добр макс зовут компан китобизнес удобн говор,"introducing=True, greeting=True",максим


__d) Извлекаем название компании__

In [15]:
COMPANY = rule(
    gram('Abbr').optional(),
    gram('ADJF').optional(),
    gram('NOUN'),
    gram('Abbr').optional(),
    gram('ADJF').optional()
)

NAME_COMPANY = (
    rule(eq('компания'), COMPANY)
)

R_2 = rule(NAME_COMPANY)

parser_2 = Parser(R_2)

In [16]:
data['company_name'] = None

for i in data[id_name].index:
    for match in parser_2.findall(data['text'][i]):
        res = [_.value for _ in match.tokens]
        data['company_name'][i] = " ".join(res[1:])

In [17]:
data[id_name]

,dlg_id,line_n,role,text,stem_processed_text,insight,name,company_name
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True,ангелина,диджитал бизнес
111,1,2,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True,ангелина,диджитал бизнес
167,2,3,client,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелин компан диджита бизнес звон повод...,introducing=True,ангелина,диджитал бизнес
251,3,2,client,Добрый меня максим зовут компания китобизнес у...,добр макс зовут компан китобизнес удобн говор,"introducing=True, greeting=True",максим,китобизнес


__e) Извлекаем реплики, где менеджер попрощался__

In [28]:
id_farewell = data['stem_processed_text'].str.contains('свидан|(хорош (вечер|дн))')
data.insight[id_farewell] = 'farewell=True'
data[id_farewell]

,dlg_id,line_n,role,text,stem_processed_text,insight,name,company_name
108,0,108,client,Всего хорошего до свидания,хорош свидан,farewell=True,None,None
163,1,54,client,До свидания,свидан,farewell=True,None,None
301,3,52,manager,Да до свидания,свидан,farewell=True,None,None
335,4,33,client,Во вторник все ну с вами да тогда до вторника ...,вторник вам вторник свидан,farewell=True,None,None
336,4,34,manager,Угу да до свидания,свидан,farewell=True,None,None
479,5,142,client,Ну до свидания хорошего вечера,свидан хорош вечер,farewell=True,None,None


__f) Проверяем требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»__

In [136]:
i = int(input('Введите номер диалога:'))

cond = (data.dlg_id == i) & (data.role == 'client')

req = sum(data['insight'][cond].isin(['greeting=True', 'farewell=True']))

if req >= 2:
    print('Менеджер выполнил все условия')
else: print('Менеджер не выполнил одно из условий')

Введите номер диалога:5
Менеджер не выполнил одно из условий
